In [3]:
import pandas as pd
result_table = "../data/pilot16_paraphrase/webapp_perspectiveparaphrase.csv"
persps = "../data/database_output/re-step1/webapp_perspective.csv"

rdf = pd.read_csv(result_table)
pdf = pd.read_csv(persps)

rdf.info()
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709 entries, 0 to 2708
Data columns (total 5 columns):
id                2709 non-null int64
perspective_id    2709 non-null int64
user_generated    2709 non-null object
session_ids       2709 non-null object
hints             2709 non-null object
dtypes: int64(2), object(3)
memory usage: 105.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20798 entries, 0 to 20797
Data columns (total 7 columns):
id                       20798 non-null int64
source                   20798 non-null object
title                    20797 non-null object
pilot1_high_agreement    20798 non-null int64
similar_persps           20798 non-null object
more_than_two_tokens     20798 non-null int64
pilot1_have_stance       20798 non-null int64
dtypes: int64(4), object(3)
memory usage: 1.1+ MB


In [6]:
rdf = rdf.loc[rdf.user_generated != "[]"]

merged = pd.merge(rdf[["perspective_id", "user_generated"]], pdf[["id", "title"]], left_on="perspective_id", right_on="id")
merged.info()

result_table = "../data/pilot16_paraphrase/result.csv"

merged.to_csv(result_table)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 0 to 114
Data columns (total 4 columns):
perspective_id    115 non-null int64
user_generated    115 non-null object
id                115 non-null int64
title             115 non-null object
dtypes: int64(2), object(2)
memory usage: 4.5+ KB
